Stack exchange
Pros
the problems are highly realistic/human like
have multiple references + a builtin rating/quality system because of upvotes
tagged with useful information
 
Cons
it’s easy to lookup
rate limits 10k per day
 
Hugging face
Pros
massive
multilingual
real code

Cons
quality depends on our corruption procedure
maybe lookup
some garbage in the dataset
kinda contrived (it’s more like a fill in the blanks dataset rather than true generation or debugging)
 
DIY
Pros
way more control over the type of problem (eg python, data analysis, pandas)

Cons
probably only good at python and maybe JavaScript
slow
maybe wrong


Testing DIY creation of debug challenges based on LLM

In [1]:
from prompting.llms import vLLMPipeline
model_id = "casperhansen/llama-3-70b-instruct-awq"
device = 'cuda'
gpus = 1
max_allowed_memory_in_gb = 79
pipeline = vLLMPipeline(device=device, model_id=model_id, gpus=gpus, llm_max_allowed_memory_in_gb=max_allowed_memory_in_gb)
# Create a vLLM_LLM instance to try and create challenges for coding
from prompting.llms import vLLM_LLM
SYSTEM_PROMPT = """You are Buggy, an AI model designed to help students learn how to debug and fix code by introducing large and significant bugs into their codebase. Your purpose is to modify provided code snippets by inserting various types of errors, including but not limited to:

1. Syntax Errors: Missing semicolons, incorrect indentation, mismatched brackets, etc.
2. Logical Errors: Incorrect conditions in loops or if statements, wrong variable assignments, etc.
3. Runtime Errors: Null pointer exceptions, divide by zero, infinite loops, etc.
4. Semantic Errors: Using incorrect algorithms or data structures for the given problem.

When introducing bugs, consider the following guidelines:

- Ensure that the inserted bugs vary in complexity to provide a range of learning opportunities.
- Maintain the overall structure and readability of the code to allow students to realistically debug it.
- Document the original code and the buggy version for comparison.
- Avoid inserting overly obscure or rare bugs that would be unrealistic for the students' current learning level.

Example Workflow:

1. Original Code:
    def calculate_sum(numbers):
        total = 0
        for number in numbers:
            total += number
        return total

2. Buggy Code:
    def calculate_sum(numbers):
        total = 0
        for number in number:  # Logical Error: 'numbers' should be used instead of 'number'
            total == number  # Syntax Error: '==' should be '+='
        return totl  # Syntax Error: misspelled 'total'

Instructions:

1. Receive the original code snippet.
2. Introduce a variety of bugs according to the guidelines.
3. Return the modified, buggy code.

Your goal is to provide a challenging yet educational experience for students learning to debug and improve their coding skills.
"""
vllm = vLLM_LLM(llm_pipeline=pipeline, system_prompt=SYSTEM_PROMPT)




import ast

def extract_functions_and_loops(code: str) -> dict:
    """
    Extracts the complete definitions of all functions and loops found in the given Python code.

    Args:
    code (str): A string containing Python code.

    Returns:
    dict: A dictionary with keys 'functions' and 'loops', each containing a list of strings.
    """
    tree = ast.parse(code)

    results = {
        'functions': [],
        'loops': []
    }

    class FunctionAndLoopVisitor(ast.NodeVisitor):
        def __init__(self):
            self.function_strings = []
            self.loop_nodes = []

        def visit_FunctionDef(self, node):
            start_lineno = node.lineno - 1  # Line numbers are 1-indexed
            end_lineno = node.end_lineno if hasattr(node, 'end_lineno') else node.body[-1].lineno
            function_lines = code.splitlines()[start_lineno:end_lineno]
            self.function_strings.append("\n".join(function_lines))
            self.generic_visit(node)

        def visit_For(self, node):
            self.loop_nodes.append(node)
            self.generic_visit(node)
        
        def visit_While(self, node):
            self.loop_nodes.append(node)
            self.generic_visit(node)

        def extract_loops(self):
            loops = []
            for node in self.loop_nodes:
                loop_code = ast.get_source_segment(code, node)
                loops.append(loop_code)
            return loops

    visitor = FunctionAndLoopVisitor()
    visitor.visit(tree)

    results['functions'] = visitor.function_strings
    results['loops'] = visitor.extract_loops()

    return results

# Example usage
input_code = """
def example_function():
    for i in range(10):
        print(i)
        for j in range(5):
            print(j)
    while x < 5:
        x += 1
"""

extracted = extract_functions_and_loops(input_code)
print('Functions:', extracted['functions'])
print('Loops:', extracted['loops'])


/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/workspace/prompting/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/workspace/prompting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-30 15:07:22,201	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-30 15:07:22,275	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip insta

2024-05-30 15:07:22.929 | 

INFO:bittensor: - Total gpu memory 84.97 GB - 


      INFO       |  - Available free memory: 84.53 GB - 


INFO:bittensor: - 93.0% of the GPU memory will be utilized for loading the model to device "cuda". - 


2024-05-30 15:07:22.934 | 

/workspace/prompting/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


      INFO       |  - Total gpu memory 84.97 GB - 
2024-05-30 15:07:22.936 |       INFO       |  - 93.0% of the GPU memory will be utilized for loading the model to device "cuda". - 
WARNING 05-30 15:07:23 config.py:173] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-30 15:07:23 llm_engine.py:73] Initializing an LLM engine with config: model='casperhansen/llama-3-70b-instruct-awq', tokenizer='casperhansen/llama-3-70b-instruct-awq', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, seed=0)
INFO 05-30 15:07:23 tokenizer.py:32] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-30 15:07:51 llm_engine.py:222] # GPU blocks: 5216, # CPU blocks: 819
Functions: ['def example_function():\n    for i in range(10):\n        print(i)\n        for j in range(5):\n            print(j)\n    while x < 5:\n        x += 1']
Loops: ['for i in range(10):\n        print(i)\n        for j in range(5):\n            print(j)', 'for j in range(5):\n            print(j)', 'while x < 5:\n        x += 1']


In [2]:
# Set the log level to ERROR to avoid excessive logging
import logging
logging.getLogger().setLevel(logging.ERROR)

# Create a dataframe with data.random['content'] as the first column and 1000 rows to start with
import pandas as pd
from prompting.tools.datasets.code import  HFCodingDataset
data = HFCodingDataset(seed=777)
query_template = """Add a large bug to the following code: \n```{code}```. This bug should prevent the code from running, and may involve changing multiple lines. 
The bugged code should still be fixable with no outside information. Make sure that the code is surrounded by ``` ```"""
df = pd.DataFrame(columns=['context', 'query', 'challenge', 'completion'])
frame_list = []
for i in range(100):
    while True:
        try:
            temp = data.random()
            if temp['topic'] == 'Python':
                content = temp['content']
                if any("print " in line for line in content.split("\n")):
                    continue
                
                #content = "\n".join(["pass" if "print" in line else line for line in content.split("\n")])
                break
        except:
            pass
    try:
        #for items in [[content]]:
        for items in extract_functions_and_loops(content).values():
            for item in items:
                if len(item.split()) < 10 or len(item.split()) > 6000:
                    continue
                # if a line in content has "print" in it, skip it

                context = item
                query = query_template.format(code=context)
                # Cut the query to first 8000 tokens. This is the maximum number of tokens that can be processed by the model
                #query = query.split()[:5000].join()
                print(len(query.split()))
                challenge = vllm.query(message=query)
                # Extract only the part of the challenge surrounded by ''' '''
                try:
                    challenge = challenge.split("'''")[1]
                except IndexError:
                    challenge = "challenge not surrounded by ''': " + challenge
                if len(challenge.split()) > 6000:
                    print("Challenge too long")
                    challenge = challenge.split()[:3100].join()
                completion_template = "Fix this code: \n```{code}```"
                completion = vllm.query(message=completion_template.format(code=challenge))
                frame_list.append({'context': context, 'query': query, 'challenge': challenge, 'completion': completion})
    except Exception as e:
        print(e)
# Turn frame_list, whcih is a list of dictionaries, into a pandas dataframe
df = pd.DataFrame(frame_list)
df.to_csv('Python_coding_challenges_GPT_SYSPROMPT.csv', index=False)

81
Composed prompt:  350


Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.42s/it]


Composed prompt:  377


Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.68s/it]


79
Composed prompt:  348


...

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.85s/it]


80
Composed prompt:  349


Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.67s/it]


Composed prompt:  376


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.96s/it]


In [3]:
len(frame_list)

188

In [4]:
frame_list

[{'context': 'def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n    path = None\n    if name == "last":\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_last.checkpoint\')\n    elif name == "best":\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_best.checkpoint\')\n    else:\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_epoch%d.checkpoint\'%count)\n    torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), path)',
  'query': 'Add a large bug to the following code: \n```def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n    path = None\n    if name == "last":\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_last.checkpoint\')\n    elif name == "best":\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_best.checkpoint\')\n    else:\n        path = os.path.join(checkpoints_dir, cf[\'data\'][\'exp\']+\'_e

In [5]:
pd.set_option('display.max_colwidth', None)
df

,context,query,challenge,completion
0,"def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), path)","Add a large bug to the following code: \n```def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), path)```. This bug should prevent the code from running, and may involve changing multiple lines. \nThe bugged code should still be fixable with no outside information. Make sure that the code is surrounded by ``` ```","challenge not surrounded by ''': \n\n```\n```def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""lastt"": # Logical Error: Extra 't' in ""lastt""\n path = os.path.joins(checkpoints_dir, cf['data']['exp']+'_last.checkpoint') # Syntax Error: 'os.path.joins' should be 'os.path.join'\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%countt) # Syntax Error: Extra 't' in 'countt'\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), pth) # Syntax Error: 'pth' should be 'path'\n model.save_checkpoint(epoch = count, optimizer=optimizer) # Runtime Error: Unnecessary function call\n```\nThis bugged code includes several errors:\n\n1. Logical Error: The `if` condition checks for `""lastt""` instead of `""last""`.\n2. Syntax Error: `os.path.joins","\n\nHere is the corrected code:\n```\ndef save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': count}, path)\n # Removed unnecessary function call\n```\nChanges made:\n\n* Fixed the logical error by changing the `if` condition to check for `""last""` instead of `""lastt""`.\n* Corrected the syntax error by replacing `os.path.joins` with `os.path.join`.\n* Removed the extra 't' in `countt` to fix the syntax error.\n* Changed `pth` to `path` to fix the syntax error.\n* Removed the unnecessary function call `model.save_checkpoint(epoch = count, optimizer=optimizer)` to fix the"
1,"def make_dir(checkpoints_dir ,cf):\n inference_path = ""./test""\n if not os.path.exists(checkpoints_dir):\n os.makedirs(checkpoints_dir)\n if not os.path.exists(cf[""data""][""log""]):\n os.makedirs(cf[""data""][""log""])\n if not os.path.exists(inference_path):\n os.makedirs(inference_path)\n \n inference_path = os.path.join(inference_path, cf['data']['exp'])\n if not os.path.exists(inference_path):\n os.makedirs(inference_path)\n checkpoints_dir = os.path.join(checkpoints_dir, cf['data']['exp'])\n if not os.path.exists(checkpoints_dir):\n os.makedirs(checkpoints_dir)\n return checkpoints_dir","Add a large bug to the following code: \n```def make_dir(checkpoints_dir ,cf):\n inference_path = ""./test""\n if not os.path.exists(checkpoints_dir):\n os.makedirs(checkpoints_dir)\n if not os

In [11]:
import pandas as pd 
import ast

# Read in the csv 
df = pd.read_csv('Python_coding_challenges_GPT_SYSPROMPT.csv')

# Use ast to check if the code is valid 
def is_valid_python(code: str) -> bool:
    """
    Checks if the given code is valid Python code.

    Args:
    code (str): A string containing Python code.

    Returns:
    bool: True if the code is valid Python code, False otherwise.
    """
    if "```" in code:
        try:
            code = code.split("```")[1].strip("```")
        except:
            pass
 
    try:
        ast.parse(code)
    except SyntaxError:
        print(code)
        return False
    return True

df['context_is_valid'] = df['context'].apply(is_valid_python)
df['challenge_is_valid'] = df['challenge'].apply(is_valid_python)
df['completion_is_valid'] = df['completion'].apply(is_valid_python)

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.cnt = 0
    def update(self, val, n=1):
        self.sum += val * n
        self.cnt += n
        if self.cnt > 0:
            self.avg = self.sum / self.cnt
    def __init__(self, source, source_dir, source_tag=None, source_commit=None,
                 source_branch=None, source_depth=None, source_checksum=None,
                 silent=False):
        super().__init__(source, source_dir, source_tag, source_commit,
                         source_branch, source_depth, source_checksum, 'svn')
        if source_tag:
            if source_branch:
                raise errors.IncompatibleOptionsError(
                    "Can't specify source-tag OR source-branch for a "
                    "Subversion source")
            else:
                raise errors.IncompatibleOptionsError(
                    "Can't specify source-tag for a Subversion source")
        elif source_branch:
            raise errors.Incompat

In [12]:
df.describe()

,context,query,challenge,completion,context_is_valid,challenge_is_valid,completion_is_valid
count,188,188,188,188,188,188,188
unique,188,188,188,188,2,2,2
top,"def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), path)","Add a large bug to the following code: \n```def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), path)```. This bug should prevent the code from running, and may involve changing multiple lines. \nThe bugged code should still be fixable with no outside information. Make sure that the code is surrounded by ``` ```","challenge not surrounded by ''': \n\n```\n```def save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""lastt"": # Logical Error: Extra 't' in ""lastt""\n path = os.path.joins(checkpoints_dir, cf['data']['exp']+'_last.checkpoint') # Syntax Error: 'os.path.joins' should be 'os.path.join'\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%countt) # Syntax Error: Extra 't' in 'countt'\n torch.save(model.save_checkpoint(epoch = count, optimizer=optimizer), pth) # Syntax Error: 'pth' should be 'path'\n model.save_checkpoint(epoch = count, optimizer=optimizer) # Runtime Error: Unnecessary function call\n```\nThis bugged code includes several errors:\n\n1. Logical Error: The `if` condition checks for `""lastt""` instead of `""last""`.\n2. Syntax Error: `os.path.joins","\n\nHere is the corrected code:\n```\ndef save_train_info(name, checkpoints_dir, cf, model, count, optimizer = None):\n path = None\n if name == ""last"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_last.checkpoint')\n elif name == ""best"":\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_best.checkpoint')\n else:\n path = os.path.join(checkpoints_dir, cf['data']['exp']+'_epoch%d.checkpoint'%count)\n torch.save({'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': count}, path)\n # Removed unnecessary function call\n```\nChanges made:\n\n* Fixed the logical error by changing the `if` condition to check for `""last""` instead of `""lastt""`.\n* Corrected the syntax error by replacing `os.path.joins` with `os.path.join`.\n* Removed the extra 't' in `countt` to fix the syntax error.\n* Changed `pth` to `path` to fix the syntax error.\n* Removed the unnecessary function call `model.save_checkpoint(epoch = count, optimizer=optimizer)` to fix the",True,True,True
freq,1,1,1,1,102,131,154


In [8]:
172/188


0.9148936170212766

In [9]:
183/188

0.973404255319149

In [10]:
102/188

0.5425531914893617